In [ ]:
CREATE WAREHOUSE IF NOT EXISTS AIRLINE_WH 
  WAREHOUSE_SIZE = 'XSMALL' 
  AUTO_SUSPEND = 60 
  AUTO_RESUME = TRUE 
  INITIALLY_SUSPENDED = TRUE;

In [ ]:
3) Create database + schema
CREATE DATABASE IF NOT EXISTS AIRLINE_DB;
CREATE SCHEMA IF NOT EXISTS AIRLINE_DB.SILVER_GOLD;

4) Creating a role for ETL 
CREATE ROLE IF NOT EXISTS ETL_ROLE;

In [ ]:
5) Creating a user for Databricks/connector
CREATE USER IF NOT EXISTS etl_user
  PASSWORD = 'Airline_delay'  
  DEFAULT_ROLE = ETL_ROLE
  DEFAULT_WAREHOUSE = AIRLINE_WH
  MUST_CHANGE_PASSWORD = FALSE;

In [ ]:
USE ROLE ACCOUNTADMIN;

In [ ]:
GRANT ROLE ETL_ROLE TO USER kanika;

In [ ]:
SHOW GRANTS TO USER kanika;

In [ ]:
USE ROLE ETL_ROLE;
USE WAREHOUSE AIRLINE_WH;
USE DATABASE AIRLINE_DB;
USE SCHEMA SILVER_GOLD;

In [ ]:
CREATE OR REPLACE TABLE GOLD_CARRIER (
  carrier STRING,
  carrier_name STRING,
  year INTEGER,
  month INTEGER,
  total_arr_flights DOUBLE,
  total_arr_del15 DOUBLE,
  total_arr_delay_minutes DOUBLE,
  total_arr_cancelled DOUBLE,
  total_arr_diverted DOUBLE,
  sum_carrier_ct DOUBLE,
  sum_weather_ct DOUBLE,
  sum_nas_ct DOUBLE,
  sum_security_ct DOUBLE,
  sum_late_aircraft_ct DOUBLE,
  delay_rate DOUBLE,
  avg_delay_per_flight DOUBLE,
  cancel_rate DOUBLE,
  divert_rate DOUBLE,
  carrier_ct_pct DOUBLE,
  weather_ct_pct DOUBLE,
  nas_ct_pct DOUBLE,
  security_ct_pct DOUBLE,
  late_aircraft_ct_pct DOUBLE
);

In [ ]:
CREATE OR REPLACE STAGE GOLD_STAGE_SAS
  URL = 'azure://saflightdelays.blob.core.windows.net/gold-parquet'
  CREDENTIALS = (AZURE_SAS_TOKEN='')
  FILE_FORMAT = (TYPE = PARQUET);

In [ ]:
LIST @GOLD_STAGE_SAS;

In [ ]:
load all parquet files under the stage into your GOLD_CARRIER table
COPY INTO GOLD_CARRIER
FROM @GOLD_STAGE_SAS
PATTERN = '.*carrier.*\.parquet'
FILE_FORMAT = (TYPE = PARQUET)
MATCH_BY_COLUMN_NAME = 'CASE_INSENSITIVE'
ON_ERROR = 'CONTINUE';

In [ ]:
SELECT * 
FROM GOLD_CARRIER 


In [ ]:
CREATE OR REPLACE TABLE GOLD_MONTHLY (
year INTEGER,
month INTEGER,
year_month STRING,
total_arr_flights DOUBLE,
total_arr_del15 DOUBLE,
total_arr_delay_minutes DOUBLE,
total_arr_cancelled DOUBLE,
total_arr_diverted DOUBLE,
sum_carrier_ct DOUBLE,
sum_weather_ct DOUBLE,
sum_nas_ct DOUBLE,
sum_security_ct DOUBLE,
sum_late_aircraft_ct DOUBLE,
delay_rate DOUBLE,
avg_delay_per_flight DOUBLE,
cancel_rate DOUBLE,
divert_rate DOUBLE,
carrier_ct_pct DOUBLE,
weather_ct_pct DOUBLE,
nas_ct_pct DOUBLE,
security_ct_pct DOUBLE,
late_aircraft_ct_pct DOUBLE
  
);

In [ ]:
COPY INTO GOLD_MONTHLY
FROM @GOLD_STAGE_SAS
PATTERN = '.*monthly.*\.parquet'       
FILE_FORMAT = (TYPE = PARQUET)
MATCH_BY_COLUMN_NAME = 'CASE_INSENSITIVE' 
ON_ERROR = 'CONTINUE';

In [ ]:
SELECT * FROM GOLD_MONTHLY

In [ ]:
CREATE OR REPLACE TABLE GOLD_CAUSES (
carrier STRING,
carrier_name STRING,
year INTEGER,
month INTEGER,
sum_carrier_ct DOUBLE,
sum_weather_ct DOUBLE,
sum_nas_ct DOUBLE,
sum_security_ct DOUBLE,
sum_late_aircraft_ct DOUBLE,
total_cause_minutes DOUBLE,
carrier_ct_pct DOUBLE,
weather_ct_pct DOUBLE,
nas_ct_pct DOUBLE,
security_ct_pct DOUBLE,
late_aircraft_ct_pct DOUBLE
  
);

In [ ]:
COPY INTO GOLD_CAUSES
FROM @GOLD_STAGE_SAS
PATTERN = '.*causes.*\.parquet'          
FILE_FORMAT = (TYPE = PARQUET)
MATCH_BY_COLUMN_NAME = 'CASE_INSENSITIVE'  
ON_ERROR = 'CONTINUE';

In [ ]:
SELECT * FROM GOLD_CAUSES LIMIT 5 

In [ ]:
CREATE OR REPLACE TABLE GOLD_MASTER (
    year INTEGER,
    month INTEGER,
    carrier STRING,
    carrier_name STRING,
    airport STRING,
    airport_name STRING,
    arr_flights DOUBLE,
    arr_del15 DOUBLE,
    carrier_ct DOUBLE,
    weather_ct DOUBLE,
    nas_ct DOUBLE,
    security_ct DOUBLE,
    late_aircraft_ct DOUBLE,
    arr_cancelled DOUBLE,
    arr_diverted DOUBLE,
    arr_delay DOUBLE,
    carrier_delay DOUBLE,
    year_month STRING,
    delay_rate DOUBLE,
    avg_delay_per_flight DOUBLE,
    cancel_rate DOUBLE,
    divert_rate DOUBLE,
    cause_total DOUBLE,
    carrier_ct_pct DOUBLE,
    weather_ct_pct DOUBLE,
    nas_ct_pct DOUBLE,
    security_ct_pct DOUBLE,
    late_aircraft_ct_pct DOUBLE
);

 

In [ ]:
COPY INTO GOLD_MASTER
FROM @GOLD_STAGE_SAS
PATTERN = '.*master.*\.parquet'         
FILE_FORMAT = (TYPE = PARQUET)
MATCH_BY_COLUMN_NAME = 'CASE_INSENSITIVE'  
ON_ERROR = 'CONTINUE';

In [ ]:
select * from  GOLD_MASTER

In [ ]:
Context 
USE ROLE <role_with_permissions>;
USE WAREHOUSE <your_wh>;
USE DATABASE <your_db>;
USE SCHEMA <your_schema>;

1) Single-row overall KPI view 

CREATE OR REPLACE VIEW v_overall_kpis AS
SELECT
  MAX(year) AS latest_year,
  MAX(month) AS latest_month,
  SUM(total_arr_flights)            AS total_arrivals,
  SUM(total_arr_del15)              AS total_del15,
  SUM(total_arr_delay_minutes)      AS total_delay_minutes,
  SUM(total_arr_cancelled)          AS total_cancelled,
  SUM(total_arr_diverted)           AS total_diverted,
  CASE WHEN SUM(total_arr_flights) = 0 THEN 0
       ELSE SUM(total_arr_del15) / NULLIF(SUM(total_arr_flights),0) END AS delay_rate,
  CASE WHEN SUM(total_arr_flights) = 0 THEN 0
       ELSE SUM(total_arr_delay_minutes) / NULLIF(SUM(total_arr_flights),0) END AS avg_delay_per_flight
FROM GOLD_MONTHLY;


2) Monthly trend view (ordered)
 Useful for line charts: year_month, total_arr_flights, delay_rate, avg_delay_per_flight

CREATE OR REPLACE VIEW v_monthly_trend AS
SELECT
  year,
  month,
  year_month,
  total_arr_flights,
  total_arr_del15,
  total_arr_delay_minutes,
  CASE WHEN total_arr_flights = 0 THEN 0 ELSE total_arr_del15 / NULLIF(total_arr_flights,0) END AS delay_rate,
  CASE WHEN total_arr_flights = 0 THEN 0 ELSE total_arr_delay_minutes / NULLIF(total_arr_flights,0) END AS avg_delay_per_flight
FROM GOLD_MONTHLY
ORDER BY year, month;


3) Top carriers summary 

CREATE OR REPLACE VIEW v_top_carriers AS
SELECT
  carrier,
  carrier_name,
  SUM(total_arr_flights)           AS total_flights,
  SUM(total_arr_del15)             AS total_del15,
  SUM(total_arr_delay_minutes)     AS total_delay_minutes,
  CASE WHEN SUM(total_arr_flights)=0 THEN 0 ELSE SUM(total_arr_del15) / NULLIF(SUM(total_arr_flights),0) END AS delay_rate,
  CASE WHEN SUM(total_arr_flights)=0 THEN 0 ELSE SUM(total_arr_delay_minutes) / NULLIF(SUM(total_arr_flights),0) END AS avg_delay_per_flight
FROM GOLD_CARRIER
GROUP BY carrier, carrier_name
ORDER BY total_flights DESC;


4) Causes percentage per carrier/month

CREATE OR REPLACE VIEW v_causes_pct AS
SELECT
  carrier,
  carrier_name,
  year,
  month,
  sum_carrier_ct,
  sum_weather_ct,
  sum_nas_ct,
  sum_security_ct,
  sum_late_aircraft_ct,
  total_cause_minutes,
  CASE WHEN total_cause_minutes = 0 THEN 0 ELSE sum_carrier_ct / NULLIF(total_cause_minutes,0) END AS carrier_ct_pct_calc,
  CASE WHEN total_cause_minutes = 0 THEN 0 ELSE sum_weather_ct / NULLIF(total_cause_minutes,0) END AS weather_ct_pct_calc,
  CASE WHEN total_cause_minutes = 0 THEN 0 ELSE sum_nas_ct / NULLIF(total_cause_minutes,0) END AS nas_ct_pct_calc,
  CASE WHEN total_cause_minutes = 0 THEN 0 ELSE sum_security_ct / NULLIF(total_cause_minutes,0) END AS security_ct_pct_calc,
  CASE WHEN total_cause_minutes = 0 THEN 0 ELSE sum_late_aircraft_ct / NULLIF(total_cause_minutes,0) END AS late_aircraft_ct_pct_calc
FROM GOLD_CAUSES
ORDER BY year, month, carrier;


 5) Master cleaned preview (safe casts and derived columns)
   Keeps original names exactly as in GOLD_MASTER but provides safe numeric casts

CREATE OR REPLACE VIEW v_master_clean AS
SELECT
  TRY_CAST(year AS INTEGER)                         AS year,
  TRY_CAST(month AS INTEGER)                        AS month,
  carrier,
  carrier_name,
  airport,
  airport_name,
  TRY_CAST(arr_flights AS DOUBLE)                   AS arr_flights,
  TRY_CAST(arr_del15 AS DOUBLE)                     AS arr_del15,
  TRY_CAST(carrier_ct AS DOUBLE)                    AS carrier_ct,
  TRY_CAST(weather_ct AS DOUBLE)                    AS weather_ct,
  TRY_CAST(nas_ct AS DOUBLE)                        AS nas_ct,
  TRY_CAST(security_ct AS DOUBLE)                   AS security_ct,
  TRY_CAST(late_aircraft_ct AS DOUBLE)              AS late_aircraft_ct,
  TRY_CAST(arr_cancelled AS DOUBLE)                 AS arr_cancelled,
  TRY_CAST(arr_diverted AS DOUBLE)                  AS arr_diverted,
  TRY_CAST(arr_delay AS DOUBLE)                     AS arr_delay,
  TRY_CAST(carrier_delay AS DOUBLE)                 AS carrier_delay,
  year_month,
  TRY_CAST(delay_rate AS DOUBLE)                    AS delay_rate,
  TRY_CAST(avg_delay_per_flight AS DOUBLE)          AS avg_delay_per_flight,
  TRY_CAST(cancel_rate AS DOUBLE)                   AS cancel_rate,
  TRY_CAST(divert_rate AS DOUBLE)                   AS divert_rate,
  TRY_CAST(cause_total AS DOUBLE)                   AS cause_total,
  TRY_CAST(carrier_ct_pct AS DOUBLE)                AS carrier_ct_pct,
  TRY_CAST(weather_ct_pct AS DOUBLE)                AS weather_ct_pct,
  TRY_CAST(nas_ct_pct AS DOUBLE)                    AS nas_ct_pct,
  TRY_CAST(security_ct_pct AS DOUBLE)               AS security_ct_pct,
  TRY_CAST(late_aircraft_ct_pct AS DOUBLE)          AS late_aircraft_ct_pct
FROM GOLD_MASTER;



In [ ]:
Total arrivals (all time / filtered by year in BI)
SELECT
    SUM(total_arr_flights) AS total_arrivals
FROM
    GOLD_MONTHLY
    -- WHERE year BETWEEN :year_from AND :year_to;

In [ ]:
Delay rate (ratio)
SELECT CASE WHEN SUM(total_arr_flights)=0 THEN 0 ELSE SUM(total_arr_del15)/NULLIF(SUM(total_arr_flights),0) END AS delay_rate
FROM GOLD_MONTHLY;

In [ ]:
Avg delay per flight
SELECT CASE WHEN SUM(total_arr_flights)=0 THEN 0 ELSE SUM(total_arr_delay_minutes)/NULLIF(SUM(total_arr_flights),0) END AS avg_delay_per_flight
FROM GOLD_MONTHLY;

In [ ]:
SELECT year_month, total_arr_flights, total_arr_del15, 
       CASE WHEN total_arr_flights = 0 THEN 0 ELSE total_arr_del15/NULLIF(total_arr_flights,0) END AS delay_rate,
       CASE WHEN total_arr_flights = 0 THEN 0 ELSE total_arr_delay_minutes/NULLIF(total_arr_flights,0) END AS avg_delay_per_flight
FROM v_monthly_trend
ORDER BY year, month;

In [ ]:
SELECT carrier, carrier_name, total_flights, total_del15, delay_rate, avg_delay_per_flight
FROM v_top_carriers
ORDER BY total_flights DESC
LIMIT 20;

In [ ]:
SELECT year, month,
       SUM(sum_carrier_ct)        AS sum_carrier_ct,
       SUM(sum_weather_ct)        AS sum_weather_ct,
       SUM(sum_nas_ct)            AS sum_nas_ct,
       SUM(sum_security_ct)       AS sum_security_ct,
       SUM(sum_late_aircraft_ct)  AS sum_late_aircraft_ct,
       SUM(total_cause_minutes)   AS total_cause_minutes
FROM GOLD_CAUSES
GROUP BY year, month
ORDER BY year, month;